# 수정중

In [1]:
import os
import sys
import pandas as pd
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

naver blog 검색

In [19]:
def blog_finder(keyword) :
    # 조회 기간 설정 : 7일(days에서 수정 가능)
    # startDate=7일전 날짜, endDate=오늘 날짜
    date = datetime.now()
    startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
    endDate = (datetime.now()).strftime('%Y-%m-%d')

    # Chrome driver 환경설정 및 실행
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_argument('headless')
    driver = webdriver.Chrome(options = options)
    base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={keyword}"
    driver.get(base_url)
    time.sleep(1)
    
    # 총 검색 결과
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    search_number = soup.find(class_="search_number")
    print("총 블로그글 개수 : ", search_number.text)
    return driver, html, soup

연관 검색어 가져오기

In [7]:
def getSubKeyword(keyword) : 
    # Chrome driver 환경설정 및 실행
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_argument('headless')
    driver = webdriver.Chrome(options = options)
    base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={keyword}"
    driver.get(base_url)
    time.sleep(1)

    # 전체 html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    
    # 연관 검색어 태그 가져오기 
    sub_keywords_data = soup.select('div.related_srch > ul > li')
    sub_keywords = []
    for i in sub_keywords_data: 
        sub_keywords.append(i.get_text())

    print(sub_keywords)
    driver.close()


title/url 가져오기

In [21]:
def getBlogInfo(keyword) : 
    blog_finder(keyword)
    
    # blog url/title 가져오기
    # html = driver.page_source
    # soup = BeautifulSoup(html, "html.parser")
    
    search_number = soup.find(class_="search_number")
    total_num = search_number.text.replace("건",'').replace(",",'')
    total_num = int(total_num)
    page_num = 1
    end_page=total_num//7+1
    end_page
    contents_num = 7

    blog_title_lst = []
    blog_url_lst = []

    while contents_num == 7 : 
        search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(search_url)
        time.sleep(0.5)
        
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
        
        # 블로그 제목, url 가져오기
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

        for post in posts_data :
            title=post.get_text().replace('\n','').strip()
            href=post.attrs['href']
            blog_title_lst.append(title)
            blog_url_lst.append(href)
        
        page_num +=1
        contents_num = len(posts_data)

    print(blog_title_lst)
    print(blog_url_lst)

In [22]:
getBlogInfo("강남역")

총 블로그글 개수 :  10,612건


UnboundLocalError: local variable 'soup' referenced before assignment